## 初期設定

In [3]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.cleaning_utils as cu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 autoreload 有効化完了


In [4]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

{'project_name': 'stock_screening_preproc',
 'data_path': './data',
 'years': [2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021,
  2022,
  2023,
  2024,
  2025],
 'files': ['fy-balance-sheet.csv',
  'fy-cash-flow-statement.csv',
  'fy-profit-and-loss.csv',
  'fy-stock-dividend.csv'],
 'files_reference': ['CodeData.csv'],
 'na_values': ['-', '0', 'NaN', ''],
 'output': {'base_path': './output',
  'files': {'fy-balance-sheet.csv': 'all_fy-balance-sheet.csv',
   'fy-cash-flow-statement.csv': 'all_fy-cash-flow-statement.csv',
   'fy-profit-and-loss.csv': 'all_fy-profit-and-loss.csv',
   'fy-stock-dividend.csv': 'all_fy-stock-dividend.csv'}}}

## 本ファイルの説明
データが最低限の品質基準を満たしているかを迅速かつ網羅的に確認します。


### データのロードの確認

In [ ]:
# 財務データ 結合前
for file in settings["files"]:
    df = dl.load_data_by_files(settings["data_path"], settings["years"], file, [""])
    print(file, df[].shape)
    #display(df[year]["コード"].unique())
    display(df[year]["コード"].nunique())
    display(df[year]["年度"].nunique())

(3034, 8)

3034

12

(3101, 8)

3101

...

3574

12

(3846, 8)

3846

31

In [ ]:
# 財務データ ファイルごとの結合
df_DATAs_BY_FILEs = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"],)
for filename in df_DATAs_BY_FILEs:
    print(filename, df_DATAs_BY_FILEs[filename].shape)
    display(df_DATAs_BY_FILEs[filename])
    display(df_DATAs_BY_FILEs[filename]["コード"].unique())
    display(df_DATAs_BY_FILEs[filename]["年度"].nunique())

fy-balance-sheet.csv (63377, 10)


,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率
0,1301,2010/03,6.430100e+10,1.853800e+10,1.839100e+10,1.258900e+10,2.672400e+10,2.712000e+09,1651.87,28.1
1,1332,2010/03,3.839240e+11,7.956700e+10,6.976600e+10,3.253000e+10,1.146390e+11,1.069930e+11,231.06,16.0
2,1333,2010/03,1.136840e+11,2.443500e+10,NaN,NaN,NaN,NaN,NaN,21.5
3,1352,2010/03,1.196700e+10,5.439000e+09,5.268000e+09,1.561000e+09,NaN,4.970000e+08,628.95,44.0
4,1375,2010/03,3.208300e+10,6.318000e+09,6.307000e+09,3.056000e+09,NaN,7.786000e+09,1.62,19.0
...,...,...,...,...,...,...,...,...,...,...
63372,9993,2025/02,5.557700e+10,2.596900e+10,2.583000e+10,2.145900e+10,8.100000e+09,3.620000e+09,2407.43,46.7
63373,9994,2025/03,6.694200e+10,3.733000e+10,3.472400e+10,2.567000e+10,3.100000e+09,2.968000e+09,3251.97,52.7
63374,9995,2024/03,4.905900e+10,2.659300e+10,2.458000e+10,1.370700e+10,1.152400e+10,NaN,901.34,54.2
63375,9996,2025/03,3.559724e+10,2.626856e+10,2.616686e+10,2.431879e+10,6.830000e+08,NaN,3149.63,73.8


array(['1301', '1332', '1333', ..., '431A', '436A', '5016'],
      shape=(4461,), dtype=object)

188

fy-cash-flow-statement.csv (61550, 8)


...

188

fy-stock-dividend.csv (55079, 8)


,コード,年度,一株配当,剰余金の配当,自社株買い,配当性向,総還元性向,純資産配当率
0,1301,2010/03,50.0,5.290000e+08,NaN,48.71,48.7,NaN
1,1332,2010/03,10.0,2.764000e+09,3000000.0,6281.82,6288.6,NaN
2,1352,2010/03,20.0,1.670000e+08,NaN,73.57,NaN,NaN
3,1375,2010/03,0.1,2.970000e+08,264000000.0,25.98,49.1,NaN
4,1376,2010/05,19.0,1.982400e+08,277000.0,20.20,20.2,NaN
...,...,...,...,...,...,...,...,...
55074,9993,2025/02,27.0,2.910000e+08,NaN,NaN,NaN,1.1
55075,9994,2025/03,54.0,5.960000e+08,NaN,16.10,NaN,1.7
55076,9995,2023/03,12.0,3.120000e+08,NaN,35.10,35.1,1.4
55077,9996,2025/03,45.0,3.502930e+08,196000.0,28.50,28.5,1.5


array(['1301', '1332', '1352', ..., '7409', '7709', '9168'],
      shape=(4275,), dtype=object)

188

In [ ]:
# 財務データ　全結合
df_ALL_DATAs = dl.merge_all_data(df_DATAs_BY_FILEs)
print(df_ALL_DATAs.shape)
#display(df_ALL_DATAs)
display(df_ALL_DATAs["コード"].unique())
display(df_ALL_DATAs["年度"].nunique())

(78079, 29)


array(['1301', '130A', '1332', ..., '9995', '9996', '9997'],
      shape=(4461,), dtype=object)

188

In [ ]:
# 上場企業の情報
df = dl.load_code_list_info(settings["data_path"], settings["files_reference"][0])
print(settings["files_reference"][0], df.shape)
display(df)

CodeData.csv (4403, 10)


,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分
0,20250930,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20250930,1305,ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算型）,ETF・ETN,-,-,-,-,-,-
2,20250930,1306,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF・ETN,-,-,-,-,-,-
3,20250930,1308,上場インデックスファンドＴＯＰＩＸ,ETF・ETN,-,-,-,-,-,-
4,20250930,1309,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF・ETN,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...
4398,20250930,9991,ジェコス,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4399,20250930,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,-,-
4400,20250930,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,-,-
4401,20250930,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,-,-


### データ型の整合性チェック

### 初期品質の概要把握